<a href="https://colab.research.google.com/github/AkashSoren1234/my-first-blog/blob/master/bertTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/filter_selected_y.csv"
df1 = pd.read_csv(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
df1.rename(columns = {'Review':'comment_text'}, inplace = True)

In [5]:
df1

,comment_text,defective product,battery life,offers,electricity consumption
0,product was damaged,1,0,0,0
1,"product received dented twice, i am not happy",1,0,0,0
2,the one which we ordered has a water outlet le...,1,0,0,0
3,product damage... speaker not working i want t...,1,0,0,0
4,but also received faulty product.,1,0,0,0
...,...,...,...,...,...
4445,The 10 Watt output is good for this tiny TV.,0,0,0,1
4446,This TV takes 65 Watts while similar size of o...,0,0,0,1
4447,"Good cooling, value for money as EER is very g...",0,0,0,1
4448,The amount of electricity is consume is nothin...,0,0,0,1


In [6]:
df1.columns

Index(['comment_text', 'defective product', 'battery life', 'offers',
       'electricity consumption'],
      dtype='object')

In [7]:
!pip install -q transformers

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 1.1MB 37.1MB/s 
     |████████████████████████████████| 3.0MB 31.8MB/s 


In [8]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [9]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [10]:
df1['list'] = df1[df1.columns[1:]].values.tolist()
new_df1 = df1[['comment_text', 'list']].copy()
new_df1.head()

,comment_text,list
0,product was damaged,"[1, 0, 0, 0]"
1,"product received dented twice, i am not happy","[1, 0, 0, 0]"
2,the one which we ordered has a water outlet le...,"[1, 0, 0, 0]"
3,product damage... speaker not working i want t...,"[1, 0, 0, 0]"
4,but also received faulty product.,"[1, 0, 0, 0]"


In [11]:
new_df1

,comment_text,list
0,product was damaged,"[1, 0, 0, 0]"
1,"product received dented twice, i am not happy","[1, 0, 0, 0]"
2,the one which we ordered has a water outlet le...,"[1, 0, 0, 0]"
3,product damage... speaker not working i want t...,"[1, 0, 0, 0]"
4,but also received faulty product.,"[1, 0, 0, 0]"
...,...,...
4445,The 10 Watt output is good for this tiny TV.,"[0, 0, 0, 1]"
4446,This TV takes 65 Watts while similar size of o...,"[0, 0, 0, 1]"
4447,"Good cooling, value for money as EER is very g...","[0, 0, 0, 1]"
4448,The amount of electricity is consume is nothin...,"[0, 0, 0, 1]"


In [12]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation='longest_first'
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
             #'comment_text': comment_text,
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [81]:
train_size = 0.7
train_dataset=new_df1.sample(frac=train_size,random_state=200)
test_dataset=new_df1.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df1.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (4450, 2)
TRAIN Dataset: (3115, 2)
TEST Dataset: (1335, 2)


In [15]:
type(testing_set)

__main__.CustomDataset

In [82]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [83]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 4)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [84]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [85]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [86]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [87]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.7849440574645996


In [88]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    #comment_text=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            #comment_text = data['comment_text']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            #comment_text.extend(comment_text)
    return fin_outputs, fin_targets

In [89]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    #print(f"comment_text = {comment_text}")
    print(f"Outputs = {outputs}")
    print(f"target = {targets}")

Accuracy Score = 0.9640449438202248
F1 Score (Micro) = 0.9674279296143767
F1 Score (Macro) = 0.7275699797748278
Outputs = [[False False  True False]
 [ True False False False]
 [ True False False False]
 ...
 [False False False  True]
 [ True False False False]
 [ True False False False]]
target = [[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.

In [90]:
test_dataset

,comment_text,list
0,the one which we ordered has a water outlet le...,"[1, 0, 0, 0]"
1,worst productthey had send me damaged tv,"[1, 0, 0, 0]"
2,my tv is damage pice not returns and refund wi...,"[1, 0, 0, 0]"
3,product damage,"[1, 0, 0, 0]"
4,"on 23rd october the tv wifi had issues, i call...","[1, 0, 0, 0]"
...,...,...
1330,I bought this 3 months ago till now its workin...,"[0, 0, 0, 1]"
1331,not sure Watt capacity information on OEM only...,"[0, 0, 0, 1]"
1332,and this showed almost 1.7 kWh for 8 hrs of usage,"[0, 0, 0, 1]"
1333,For example an AC running at 1400 Watts will h...,"[0, 0, 0, 1]"


In [91]:
predicted_list=[]
for i in range(len(test_dataset)):
    a = test_dataset['comment_text'][i]
    b = test_dataset['list'][i]
    data =[[a,b]]
    dfx = pd.DataFrame(data, columns = ['comment_text', 'list'])
    new_set = CustomDataset(dfx, tokenizer, MAX_LEN)
    new_loader = DataLoader(new_set, **test_params)
    def prediction(epoch):
        model.eval()
        fin_targets=[]
        fin_outputs=[]
        with torch.no_grad():
            for _, data in enumerate(new_loader, 0):
                ids = data['ids'].to(device, dtype = torch.long)
                mask = data['mask'].to(device, dtype = torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
                targets = data['targets'].to(device, dtype = torch.float)
                outputs = model(ids, mask, token_type_ids)
                fin_targets.extend(targets.cpu().detach().numpy().tolist())
                fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            return fin_outputs, fin_targets

    for epoch in range(EPOCHS):
        outputs, targets = prediction(epoch)
        outputs = np.array(outputs) >= 0.4
        c = outputs.tolist()[0]
        predicted_list.append(c)
    
 

In [92]:
test=test_dataset.copy()

In [93]:
test.rename(columns = {'comment_text':'Review'}, inplace = True)

In [94]:
test = test.assign(predicted_label=predicted_list)

In [95]:
test

,comment_text,list,predicted_label
0,the one which we ordered has a water outlet le...,"[1, 0, 0, 0]","[True, False, False, False]"
1,worst productthey had send me damaged tv,"[1, 0, 0, 0]","[True, False, False, False]"
2,my tv is damage pice not returns and refund wi...,"[1, 0, 0, 0]","[True, False, False, False]"
3,product damage,"[1, 0, 0, 0]","[True, False, False, False]"
4,"on 23rd october the tv wifi had issues, i call...","[1, 0, 0, 0]","[True, False, False, False]"
...,...,...,...
1330,I bought this 3 months ago till now its workin...,"[0, 0, 0, 1]","[False, False, False, True]"
1331,not sure Watt capacity information on OEM only...,"[0, 0, 0, 1]","[False, False, False, True]"
1332,and this showed almost 1.7 kWh for 8 hrs of usage,"[0, 0, 0, 1]","[False, False, False, True]"
1333,For example an AC running at 1400 Watts will h...,"[0, 0, 0, 1]","[False, False, False, True]"


In [98]:
for i in range(len(test)):
    a = test['predicted_label'][i]
    b = [1 if x==True else 0 for x in a]
    test['predicted_label'][i] = b

In [106]:
test

,Review,actual_label,predicted_label
0,the one which we ordered has a water outlet le...,"[1, 0, 0, 0]","[1, 0, 0, 0]"
1,worst productthey had send me damaged tv,"[1, 0, 0, 0]","[1, 0, 0, 0]"
2,my tv is damage pice not returns and refund wi...,"[1, 0, 0, 0]","[1, 0, 0, 0]"
3,product damage,"[1, 0, 0, 0]","[1, 0, 0, 0]"
4,"on 23rd october the tv wifi had issues, i call...","[1, 0, 0, 0]","[1, 0, 0, 0]"
...,...,...,...
1330,I bought this 3 months ago till now its workin...,"[0, 0, 0, 1]","[0, 0, 0, 1]"
1331,not sure Watt capacity information on OEM only...,"[0, 0, 0, 1]","[0, 0, 0, 1]"
1332,and this showed almost 1.7 kWh for 8 hrs of usage,"[0, 0, 0, 1]","[0, 0, 0, 1]"
1333,For example an AC running at 1400 Watts will h...,"[0, 0, 0, 1]","[0, 0, 0, 1]"


In [100]:
test.reset_index(inplace = True) 

In [102]:
test.rename(columns = {'comment_text':'Review', 'list':'actual_label'}, inplace = True)

In [105]:
test = test[['Review','actual_label','predicted_label']]

In [107]:
index = []
for i in range(len(test)):
    a = test['actual_label'][i]
    b = test['predicted_label'][i]
    if a != b:
        index.append(i)
test = test.iloc[index]
test.reset_index(inplace = True)
test = test[['Review','actual_label','predicted_label']] 

In [110]:
test

,Review,actual_label,predicted_label
0,flipkart sells very old or defective product p...,"[1, 0, 1, 0]","[1, 0, 0, 0]"
1,nice product delivered by flipkart value for m...,"[1, 0, 1, 0]","[1, 0, 0, 0]"
2,i found damage in tv power cable while it's de...,"[1, 0, 0, 0]","[1, 0, 0, 1]"
3,when i received the product i was damaged agai...,"[1, 0, 0, 0]","[0, 0, 1, 0]"
4,"dear seller, better defective products shall b...","[1, 0, 0, 0]","[0, 0, 1, 0]"
5,got at good price but got dented.,"[1, 0, 0, 0]","[1, 0, 1, 0]"
6,"delivery good, what product performance poor, ...","[1, 0, 0, 0]","[1, 0, 0, 1]"
7,i never purchase any costly item through amazo...,"[1, 0, 0, 0]","[1, 0, 1, 0]"
8,wrong information provided for amazon acko ext...,"[0, 0, 1, 0]","[1, 0, 1, 0]"
9,product is good but cash back is cheating,"[0, 0, 1, 0]","[1, 0, 0, 0]"


In [111]:

actual_defective_product=[]
actual_battery_life=[]
actual_offers=[]
actual_electricity_consumption=[]
predicted_defective_product=[]
predicted_battery_life=[]
predicted_offers=[]
predicted_electricity_consumption=[]
for i in range(len(test)):
    a = test2['actual_label'][i]
    actual_defective_product.append(a[0])
    actual_battery_life.append(a[1])
    actual_offers.append(a[2])
    actual_electricity_consumption.append(a[3])
for i in range(len(test2)):
    b = test['predicted_label'][i]
    predicted_defective_product.append(b[0])
    predicted_battery_life.append(b[1])
    predicted_offers.append(b[2])
    predicted_electricity_consumption.append(b[3])

In [113]:
test = test.assign(**{'actual_defective_product' : actual_defective_product, 'actual_battery_life' : actual_battery_life, 'actual_offers': actual_offers, 'actual_electricity_consumption':actual_electricity_consumption, 'predicted_defective_product':predicted_defective_product, 'predicted_battery_life':predicted_battery_life, 'predicted_offers':predicted_offers, 'predicted_electricity_consumption':predicted_electricity_consumption })

In [114]:
test = test[['Review', 'actual_defective_product',
       'actual_battery_life', 'actual_offers',
       'actual_electricity_consumption', 'predicted_defective_product',
       'predicted_battery_life', 'predicted_offers',
       'predicted_electricity_consumption']]

In [118]:
test.to_csv("/content/drive/My Drive/list_format_labelling.csv")

In [117]:
test2

,Review,actual_label,predicted_label
0,flipkart sells very old or defective product p...,"[1, 0, 1, 0]","[1, 0, 0, 0]"
1,nice product delivered by flipkart value for m...,"[1, 0, 1, 0]","[1, 0, 0, 0]"
2,i found damage in tv power cable while it's de...,"[1, 0, 0, 0]","[1, 0, 0, 1]"
3,when i received the product i was damaged agai...,"[1, 0, 0, 0]","[0, 0, 1, 0]"
4,"dear seller, better defective products shall b...","[1, 0, 0, 0]","[0, 0, 1, 0]"
5,got at good price but got dented.,"[1, 0, 0, 0]","[1, 0, 1, 0]"
6,"delivery good, what product performance poor, ...","[1, 0, 0, 0]","[1, 0, 0, 1]"
7,i never purchase any costly item through amazo...,"[1, 0, 0, 0]","[1, 0, 1, 0]"
8,wrong information provided for amazon acko ext...,"[0, 0, 1, 0]","[1, 0, 1, 0]"
9,product is good but cash back is cheating,"[0, 0, 1, 0]","[1, 0, 0, 0]"
